# Create heatmap from saved radial averages

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os

In [3]:
def unique(list_to_check):  
    unique_list = [] 
    for x in list_to_check: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
    
def load_radial_data(file_prefix, fragment):
    
    load_from_path = 'intermediate_data/'
    files = [i for i in os.listdir(load_from_path) if os.path.isfile(os.path.join(load_from_path,i)) and str(file_prefix)+"_"+str(fragment) in i]
    
    radial_averages =[]
    delay_stage_pos = []
    norm_counts = []
    
    for i in files:
        npzfile = np.load(load_from_path+i)
        radial_averages.append(npzfile['radial_average'])
        delay_stage_pos.append(float(npzfile['delay_stage']))
        fragment = (npzfile['fragment'])
        norm_counts.append(float(npzfile['number_of_trains']))
        
    radial_averages = np.array(radial_averages)
    delay_stage_pos = np.array(delay_stage_pos)
    norm_counts = np.array(norm_counts)

    sort_ind = np.argsort(delay_stage_pos)
    radial_averages = radial_averages[sort_ind]
    norm_counts = norm_counts[sort_ind]
    delay_stage_pos = delay_stage_pos[sort_ind]
    
    radial_averages_norm = []
    for i in range(len(radial_averages)):
        radial_averages_norm.append(radial_averages[i]/norm_counts[i])
    radial_averages_norm = np.array(radial_averages_norm)
    

    uni_delay = np.array(unique(delay_stage_pos))
    uni_radial = []
    
    for i in range(len(uni_delay)):
        tt = []
        number_to_norm = 0
        for j in range(len(delay_stage_pos)):
            if uni_delay[i] == delay_stage_pos[j]:
                number_to_norm = number_to_norm +1
                tt.append(radial_averages_norm[j])
        uni_radial.append((np.sum(np.array(tt), axis = 0)/number_to_norm))
    uni_radial = np.array(uni_radial)
    
    print('fragment:',fragment)
    print('number of runs:',len(delay_stage_pos))
    print('number of unqique delays:',len(uni_delay))

    return uni_radial, uni_delay 

def simple_plots(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos = False, stop_delay_pos = False):
    radial_averages_norm, delay_stage_pos = slice_in_delay(radial_averages_norm, delay_stage_pos, start_delay_pos, stop_delay_pos)
    colors = cm.rainbow(np.linspace(0, 1, len(radial_averages_norm)))
    fig = plt.figure(figsize = (10,10),num = 1)
    for i in range(len(radial_averages_norm)):
        plt.plot(radial_averages_norm[i],label=delay_stage_pos[i], color = colors[i])
    plt.xlabel('r')
    plt.ylabel('counts')
    plt.legend()
    plt.title(str(fragment), fontsize = 20)
    print('Number of delay steps:',len(delay_stage_pos))
    plt.show()
    
def heatmap(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos = False, stop_delay_pos = False):
    radial_averages_norm, delay_stage_pos = slice_in_delay(radial_averages_norm, delay_stage_pos, start_delay_pos, stop_delay_pos)
    fig = plt.figure(figsize = (10,10),num = 2)
    image = np.column_stack(radial_averages_norm)
    plt.imshow(image, aspect="auto", origin="lower")
    plt.xticks(np.arange(len(delay_stage_pos)), delay_stage_pos)
    plt.title(str(fragment), fontsize = 20)
    print('Number of delay steps:',len(delay_stage_pos))
    plt.show()
    
def x_projection_heatmap(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos = False, stop_delay_pos = False):
    radial_averages_norm, delay_stage_pos = slice_in_delay(radial_averages_norm, delay_stage_pos, start_delay_pos, stop_delay_pos)
    intensities_per_delay = np.sum(radial_averages_norm,axis =1)
    fig = plt.figure(figsize = (10,10),num = 3)
    plt.plot(delay_stage_pos[1:], intensities_per_delay[1:],'o')
    plt.xlabel('delay')
    plt.ylabel('integral of radial average')
    plt.title('800nm pump-probe delay scan, fragment {}'.format(fragment))
    plt.show()

def slice_in_delay(radial_averages_norm, delay_stage_pos, start_delay_pos, stop_delay_pos):
    if start_delay_pos == False:
        start_delay_pos = np.min(delay_stage_pos)
    if stop_delay_pos == False:
        stop_delay_pos = np.max(delay_stage_pos)
    radial_averages_norm = radial_averages_norm[(delay_stage_pos >= start_delay_pos) & (delay_stage_pos <= stop_delay_pos)]
    delay_stage_pos = delay_stage_pos[(delay_stage_pos >= start_delay_pos) & (delay_stage_pos <= stop_delay_pos)] 
    return radial_averages_norm, delay_stage_pos


In [13]:
file_prefix = '400nm_Iodopyridine_I3p'
fragment = 'I3+'

start_delay_pos = False
stop_delay_pos = False

radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)

fragment: I3+
number of runs: 27
number of unqique delays: 26


### simple plots

In [6]:
simple_plots(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos, stop_delay_pos)

Number of delay steps: 17


### heatmap

In [15]:
heatmap(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos, stop_delay_pos)

Number of delay steps: 26


### x-axis projection of heatmap

In [ ]:
# start_delay_pos = 342
# stop_delay_pos = 346

x_projection_heatmap(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos, stop_delay_pos)

### check two delay scans

In [ ]:

fig = plt.figure(figsize = (10,10),num = 3)

file_prefix = '800nm_delay_scan_1'
fragment = 'I6+'
radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)
intensities_per_delay = np.sum(radial_averages_norm,axis =1)
plt.plot(delay_stage_pos, intensities_per_delay,'bo', label='scan1')
print(len(intensities_per_delay))

file_prefix = '800nm_delay_scan_2'
fragment = 'I6+'
radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)
intensities_per_delay = np.sum(radial_averages_norm,axis =1)
plt.plot(delay_stage_pos, intensities_per_delay,'ro', label='scan2')
print(len(intensities_per_delay))

file_prefix = '800nm_delay_scan'
fragment = 'I6+'
radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)
intensities_per_delay = np.sum(radial_averages_norm,axis =1)
plt.plot(delay_stage_pos, intensities_per_delay,'kx', label='all')
print(len(intensities_per_delay))


plt.xlabel('delay _pos')
plt.ylabel('integral of radial average')
plt.title('800nm pump-probe delay scan, fragment {}'.format(fragment))
plt.legend()


In [ ]:
file_prefix = '800nm_Iodopyridine_I2p'
fragment = 'I2+'

start_delay_pos = False
stop_delay_pos = False

radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)

### simple plots

In [ ]:
simple_plots(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos, stop_delay_pos)

### heatmap

In [ ]:
heatmap(radial_averages_norm, delay_stage_pos, fragment, start_delay_pos, stop_delay_pos)